# YOLOv8 Custom Training

## Các bước thực hiện:
1. Setup Google Colab environment
2. Import libraries và định nghĩa các components
3. Cấu hình dataset tùy chỉnh
4. Download và setup Roboflow dataset
5. Định nghĩa custom dataset class (không augmentation)
6. Training loop với resume capability

## 1. Setup Google Colab Environment

## 3. Cấu hình Dataset

In [1]:
# Import các thư viện cần thiết
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset as TorchDataset, DataLoader
import yaml
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import shutil
import csv

# Roboflow
try:
    from roboflow import Roboflow
except ImportError:
    os.system("pip install roboflow")
    from roboflow import Roboflow

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# ========== CẤU HÌNH CHO DATASET ==========

# Thông tin Roboflow
ROBOFLOW_API_KEY = "YxgjW7IrHbIO7qDyeTKT"
WORKSPACE_NAME = "yolo-tnj0p"
PROJECT_NAME = "wild-animals-detection-yolov8-zw3fg"
VERSION_NUMBER = 1

# Cấu hình model
NUM_CLASSES = 5                         # Số classes trong dataset
INPUT_SIZE = 640                        # Kích thước input image
MODEL_VERSION = 'n'                     # 'n', 's', 'm', 'l', 'x'

# Cấu hình training
BATCH_SIZE = 32                         # Batch size
NUM_EPOCHS = 10                       # Số epochs
LEARNING_RATE = 0.01                    # Learning rate

# Paths cho Google Colab
try:
    DRIVE_SAVE_PATH = "./working/"
    os.makedirs(DRIVE_SAVE_PATH, exist_ok=True)

    SAVE_PATH = os.path.join(DRIVE_SAVE_PATH, "custom_yolo_model.pth")
    DATASET_PATH = "./data/wild-animals-detection-yolov8"  # Fixed path

    CHECKPOINT_DIR = os.path.join(DRIVE_SAVE_PATH, "checkpoints")
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

except:
    SAVE_PATH = "./custom_yolo_model.pth"
    DATASET_PATH = "./roboflow_dataset"
    CHECKPOINT_DIR = "./checkpoints"
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

RESUME_TRAINING = True
SAVE_CHECKPOINT_EVERY = 10

## 4. YOLOv8 Architecture Components

YOLOv8 bao gồm 3 phần chính:
- **Backbone**: Trích xuất features từ ảnh đầu vào
- **Neck**: Kết hợp features ở các scales khác nhau  
- **Head**: Dự đoán bounding boxes và classifications

### 4.1. Basic Building Blocks

In [3]:
# Conv Block
class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, groups=1, activation=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False, groups=groups)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0.03)
        self.act = nn.SiLU(inplace=True) if activation else nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

# Bottleneck
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, shortcut=True):
        super().__init__()
        self.conv1 = Conv(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = Conv(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.shortcut = shortcut

    def forward(self, x):
        x_in = x
        x = self.conv1(x)
        x = self.conv2(x)
        if self.shortcut:
            x = x + x_in
        return x

# C2f Block
class C2f(nn.Module):
    def __init__(self, in_channels, out_channels, num_bottlenecks, shortcut=True):
        super().__init__()
        self.mid_channels = out_channels // 2
        self.num_bottlenecks = num_bottlenecks
        self.conv1 = Conv(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.m = nn.ModuleList([Bottleneck(self.mid_channels, self.mid_channels) for _ in range(num_bottlenecks)])
        self.conv2 = Conv((num_bottlenecks + 2) * out_channels // 2, out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.conv1(x)
        x1, x2 = x[:, :x.shape[1]//2, :, :], x[:, x.shape[1]//2:, :, :]
        outputs = [x1, x2]
        for i in range(self.num_bottlenecks):
            x1 = self.m[i](x1)
            outputs.insert(0, x1)
        outputs = torch.cat(outputs, dim=1)
        out = self.conv2(outputs)
        return out

# SPPF Block
class SPPF(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5):
        super().__init__()
        hidden_channels = in_channels // 2
        self.conv1 = Conv(in_channels, hidden_channels, kernel_size=1, stride=1, padding=0)
        self.conv2 = Conv(4 * hidden_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.m = nn.MaxPool2d(kernel_size=kernel_size, stride=1, padding=kernel_size//2, dilation=1, ceil_mode=False)

    def forward(self, x):
        x = self.conv1(x)
        y1 = self.m(x)
        y2 = self.m(y1)
        y3 = self.m(y2)
        y = torch.cat([x, y1, y2, y3], dim=1)
        y = self.conv2(y)
        return y

In [4]:
# YOLO Parameters
def yolo_params(version):
    if version == 'n':
        return 1/3, 1/4, 2.0
    elif version == 's':
        return 1/3, 1/2, 2.0
    elif version == 'm':
        return 2/3, 3/4, 1.5
    elif version == 'l':
        return 1.0, 1.0, 1.0
    elif version == 'x':
        return 1.0, 1.25, 1.0

# Backbone
class Backbone(nn.Module):
    def __init__(self, version, in_channels=3):
        super().__init__()
        d, w, r = yolo_params(version)

        # Conv layers
        self.conv_0 = Conv(in_channels, int(64*w), kernel_size=3, stride=2, padding=1)
        self.conv_1 = Conv(int(64*w), int(128*w), kernel_size=3, stride=2, padding=1)
        self.conv_3 = Conv(int(128*w), int(256*w), kernel_size=3, stride=2, padding=1)
        self.conv_5 = Conv(int(256*w), int(512*w), kernel_size=3, stride=2, padding=1)
        self.conv_7 = Conv(int(512*w), int(512*w*r), kernel_size=3, stride=2, padding=1)

        # C2f layers
        self.c2f_2 = C2f(int(128*w), int(128*w), num_bottlenecks=int(3*d), shortcut=True)
        self.c2f_4 = C2f(int(256*w), int(256*w), num_bottlenecks=int(6*d), shortcut=True)
        self.c2f_6 = C2f(int(512*w), int(512*w), num_bottlenecks=int(6*d), shortcut=True)
        self.c2f_8 = C2f(int(512*w*r), int(512*w*r), num_bottlenecks=int(3*d), shortcut=True)

        # SPPF
        self.sppf = SPPF(int(512*w*r), int(512*w*r))

    def forward(self, x):
        x = self.conv_0(x)
        x = self.conv_1(x)
        x = self.c2f_2(x)
        x = self.conv_3(x)
        out1 = self.c2f_4(x)
        x = self.conv_5(out1)
        out2 = self.c2f_6(x)
        x = self.conv_7(out2)
        x = self.c2f_8(x)
        out3 = self.sppf(x)
        return out1, out2, out3

### 4.2. Backbone - Feature Extractor

Backbone sử dụng modified CSPDarknet53 với các components:
- **Conv layers**: Giảm kích thước và tăng channels
- **C2f layers**: Cross-stage partial bottleneck
- **SPPF**: Spatial pyramid pooling fast

In [5]:
# Upsample
class Upsample(nn.Module):
    def __init__(self, scale_factor=2, mode='nearest'):
        super().__init__()
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        return nn.functional.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)

# Neck
class Neck(nn.Module):
    def __init__(self, version):
        super().__init__()
        d, w, r = yolo_params(version)

        self.up = Upsample()
        self.c2f_1 = C2f(in_channels=int(512*w*(1+r)), out_channels=int(512*w), num_bottlenecks=int(3*d), shortcut=False)
        self.c2f_2 = C2f(in_channels=int(768*w), out_channels=int(256*w), num_bottlenecks=int(3*d), shortcut=False)
        self.c2f_3 = C2f(in_channels=int(768*w), out_channels=int(512*w), num_bottlenecks=int(3*d), shortcut=False)
        self.c2f_4 = C2f(in_channels=int(512*w*(1+r)), out_channels=int(512*w*r), num_bottlenecks=int(3*d), shortcut=False)

        self.cv_1 = Conv(in_channels=int(256*w), out_channels=int(256*w), kernel_size=3, stride=2, padding=1)
        self.cv_2 = Conv(in_channels=int(512*w), out_channels=int(512*w), kernel_size=3, stride=2, padding=1)

    def forward(self, x_res_1, x_res_2, x):
        res_1 = x
        x = self.up(x)
        x = torch.cat([x, x_res_2], dim=1)
        res_2 = self.c2f_1(x)
        x = self.up(res_2)
        x = torch.cat([x, x_res_1], dim=1)
        out_1 = self.c2f_2(x)
        x = self.cv_1(out_1)
        x = torch.cat([x, res_2], dim=1)
        out_2 = self.c2f_3(x)
        x = self.cv_2(out_2)
        x = torch.cat([x, res_1], dim=1)
        out_3 = self.c2f_4(x)
        return out_1, out_2, out_3

### 4.3. Neck - Feature Pyramid Network

Neck sử dụng FPN (Feature Pyramid Network) để:
- **Upsample**: Tăng resolution của features
- **Feature fusion**: Kết hợp features từ các scales khác nhau
- **Multi-scale processing**: Xử lý objects ở nhiều kích thước

In [6]:
# DFL
class DFL(nn.Module):
    def __init__(self, ch=16):
        super().__init__()
        self.ch = ch
        self.conv = nn.Conv2d(in_channels=ch, out_channels=1, kernel_size=1, bias=False).requires_grad_(False)
        x = torch.arange(ch, dtype=torch.float).view(1, ch, 1, 1)
        self.conv.weight.data[:] = torch.nn.Parameter(x)

    def forward(self, x):
        b, c, a = x.shape
        x = x.view(b, 4, self.ch, a).transpose(1, 2)
        x = x.softmax(1)
        x = self.conv(x)
        return x.view(b, 4, a)

# Head với custom num_classes
class Head(nn.Module):
    def __init__(self, version, ch=16, num_classes=NUM_CLASSES):
        super().__init__()
        self.ch = ch
        self.coordinates = self.ch * 4
        self.nc = num_classes  # Sử dụng custom số classes
        self.no = self.coordinates + self.nc
        self.stride = torch.zeros(3)

        d, w, r = yolo_params(version=version)

        # Box detection heads
        self.box = nn.ModuleList([
            nn.Sequential(Conv(int(256*w), self.coordinates, kernel_size=3, stride=1, padding=1),
                          Conv(self.coordinates, self.coordinates, kernel_size=3, stride=1, padding=1),
                          nn.Conv2d(self.coordinates, self.coordinates, kernel_size=1, stride=1)),
            nn.Sequential(Conv(int(512*w), self.coordinates, kernel_size=3, stride=1, padding=1),
                          Conv(self.coordinates, self.coordinates, kernel_size=3, stride=1, padding=1),
                          nn.Conv2d(self.coordinates, self.coordinates, kernel_size=1, stride=1)),
            nn.Sequential(Conv(int(512*w*r), self.coordinates, kernel_size=3, stride=1, padding=1),
                          Conv(self.coordinates, self.coordinates, kernel_size=3, stride=1, padding=1),
                          nn.Conv2d(self.coordinates, self.coordinates, kernel_size=1, stride=1))
        ])

        # Classification heads
        self.cls = nn.ModuleList([
            nn.Sequential(Conv(int(256*w), self.nc, kernel_size=3, stride=1, padding=1),
                          Conv(self.nc, self.nc, kernel_size=3, stride=1, padding=1),
                          nn.Conv2d(self.nc, self.nc, kernel_size=1, stride=1)),
            nn.Sequential(Conv(int(512*w), self.nc, kernel_size=3, stride=1, padding=1),
                          Conv(self.nc, self.nc, kernel_size=3, stride=1, padding=1),
                          nn.Conv2d(self.nc, self.nc, kernel_size=1, stride=1)),
            nn.Sequential(Conv(int(512*w*r), self.nc, kernel_size=3, stride=1, padding=1),
                          Conv(self.nc, self.nc, kernel_size=3, stride=1, padding=1),
                          nn.Conv2d(self.nc, self.nc, kernel_size=1, stride=1))
        ])

        self.dfl = DFL()

    def forward(self, x):
        for i in range(len(self.box)):
            box = self.box[i](x[i])
            cls = self.cls[i](x[i])
            x[i] = torch.cat((box, cls), dim=1)

        if self.training:
            return x

        anchors, strides = (i.transpose(0, 1) for i in self.make_anchors(x, self.stride))
        x = torch.cat([i.view(x[0].shape[0], self.no, -1) for i in x], dim=2)
        box, cls = x.split(split_size=(4 * self.ch, self.nc), dim=1)
        a, b = self.dfl(box).chunk(2, 1)
        a = anchors.unsqueeze(0) - a
        b = anchors.unsqueeze(0) + b
        box = torch.cat(tensors=((a + b) / 2, b - a), dim=1)
        return torch.cat(tensors=(box * strides, cls.sigmoid()), dim=1)

    def make_anchors(self, x, strides, offset=0.5):
        assert x is not None
        anchor_tensor, stride_tensor = [], []
        dtype, device = x[0].dtype, x[0].device
        for i, stride in enumerate(strides):
            _, _, h, w = x[i].shape
            sx = torch.arange(end=w, device=device, dtype=dtype) + offset
            sy = torch.arange(end=h, device=device, dtype=dtype) + offset
            sy, sx = torch.meshgrid(sy, sx)
            anchor_tensor.append(torch.stack((sx, sy), -1).view(-1, 2))
            stride_tensor.append(torch.full((h * w, 1), stride, dtype=dtype, device=device))
        return torch.cat(anchor_tensor), torch.cat(stride_tensor)

### 4.4. Head - Detection & Classification

Head gồm 3 modules chính:
- **DFL (Distribution Focal Loss)**: Tinh chỉnh bounding box coordinates
- **Box regression**: Dự đoán vị trí và kích thước objects
- **Classification**: Dự đoán class probabilities

In [7]:
# Custom YOLOv8 Model
class CustomYolo(nn.Module):
    def __init__(self, version, num_classes=NUM_CLASSES):
        super().__init__()
        self.backbone = Backbone(version=version)
        self.neck = Neck(version=version)
        self.head = Head(version=version, num_classes=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x[0], x[1], x[2])
        return self.head(list(x))

# Test model
model = CustomYolo(version=MODEL_VERSION, num_classes=NUM_CLASSES)

### 4.5. Complete YOLOv8 Model

Kết hợp tất cả components thành model hoàn chỉnh với custom số classes.

## 5. Download Dataset từ Roboflow

## 6. Custom Dataset Class

In [8]:
class CustomDataset(TorchDataset):
    def __init__(self, image_paths, input_size=INPUT_SIZE):
        self.image_paths = image_paths
        self.input_size = input_size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load corresponding label file
        label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')

        # Parse YOLO format labels
        boxes = []
        classes = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        x_center, y_center, width, height = map(float, parts[1:5])

                        classes.append(class_id)
                        boxes.append([x_center, y_center, width, height])

        # Convert to numpy arrays
        boxes = np.array(boxes, dtype=np.float32) if boxes else np.zeros((0, 4), dtype=np.float32)
        classes = np.array(classes, dtype=np.int64) if classes else np.zeros((0,), dtype=np.int64)

        # Resize image và adjust boxes
        h, w = image.shape[:2]
        scale = self.input_size / max(h, w)
        new_h, new_w = int(h * scale), int(w * scale)

        # Resize image
        image = cv2.resize(image, (new_w, new_h))

        # Create padded image
        padded_image = np.full((self.input_size, self.input_size, 3), 114, dtype=np.uint8)
        padded_image[:new_h, :new_w] = image

        # Convert to tensor format
        image_tensor = torch.from_numpy(padded_image.transpose(2, 0, 1)).float() / 255.0

        # Create targets dictionary
        targets = {
            'cls': torch.from_numpy(classes),
            'box': torch.from_numpy(boxes),
            'idx': torch.full((len(classes),), idx, dtype=torch.long)
        }

        return image_tensor, targets

    @staticmethod
    def collate_fn(batch):
        images, targets = zip(*batch)

        # Stack images
        images = torch.stack(images, 0)

        # Concatenate targets
        cls_list = []
        box_list = []
        idx_list = []

        for i, target in enumerate(targets):
            if len(target['cls']) > 0:
                cls_list.append(target['cls'])
                box_list.append(target['box'])
                batch_idx = torch.full((len(target['cls']),), i, dtype=torch.long)
                idx_list.append(batch_idx)

        # Concatenate all targets
        all_cls = torch.cat(cls_list, 0) if cls_list else torch.zeros(0, dtype=torch.long)
        all_boxes = torch.cat(box_list, 0) if box_list else torch.zeros(0, 4)
        all_idx = torch.cat(idx_list, 0) if idx_list else torch.zeros(0, dtype=torch.long)

        batch_targets = {
            'cls': all_cls,
            'box': all_boxes,
            'idx': all_idx
        }

        return images, batch_targets

## 7. Load Dataset

In [9]:
def load_dataset_paths(dataset_path):
    train_images = []
    val_images = []

    train_dir = os.path.join(dataset_path, "train", "images")
    if os.path.exists(train_dir):
        for img_file in os.listdir(train_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                train_images.append(os.path.join(train_dir, img_file))

    val_dir = os.path.join(dataset_path, "valid", "images")
    if os.path.exists(val_dir):
        for img_file in os.listdir(val_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                val_images.append(os.path.join(val_dir, img_file))

    return train_images, val_images

# Load dataset
if os.path.exists(DATASET_PATH):
    train_images, val_images = load_dataset_paths(DATASET_PATH)
    
    if len(train_images) > 0 and len(val_images) > 0:
        train_dataset = CustomDataset(train_images, input_size=INPUT_SIZE)
        val_dataset = CustomDataset(val_images, input_size=INPUT_SIZE)

        train_loader = DataLoader(
            train_dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=2,
            collate_fn=CustomDataset.collate_fn,
            pin_memory=True if device.type == 'cuda' else False
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=2,
            collate_fn=CustomDataset.collate_fn,
            pin_memory=True if device.type == 'cuda' else False
        )
        
        print(f"Dataset loaded: {len(train_images)} train, {len(val_images)} val images")
    else:
        print("No images found. Please check dataset path.")
        train_loader = None
        val_loader = None
else:
    print("Dataset path not found.")
    train_loader = None
    val_loader = None

Dataset loaded: 8991 train, 2569 val images


In [10]:
images, targets = next(iter(train_loader))

# Kiểm tra shape của images
print(images.shape)        # [batch_size, channels, height, width]
print(len(targets))  # số object trong batch, nếu là list/dict
print(targets)

torch.Size([32, 3, 640, 640])
3
{'cls': tensor([2, 0, 2, 4, 0, 3, 1, 3, 3, 3, 0, 2, 4, 3, 0, 0, 0, 0, 1, 4, 1, 3, 1, 1,
        1, 4, 4, 3, 3, 2, 2, 2, 4, 2, 3, 1, 1, 1, 1, 1, 2, 2, 4, 4, 4, 4, 4, 4]), 'box': tensor([[0.6367, 0.4969, 0.7266, 0.9937],
        [0.6805, 0.5203, 0.6320, 0.9516],
        [0.6398, 0.5320, 0.6516, 0.9234],
        [0.7437, 0.1469, 0.1219, 0.1672],
        [0.4984, 0.4977, 0.9969, 0.9953],
        [0.7008, 0.4898, 0.4906, 0.8234],
        [0.4344, 0.5000, 0.7336, 1.0000],
        [0.7641, 0.7156, 0.4719, 0.5234],
        [0.3023, 0.6172, 0.4453, 0.4062],
        [0.4461, 0.2313, 0.8922, 0.4219],
        [0.4938, 0.5109, 0.7117, 0.8555],
        [0.4305, 0.5070, 0.6992, 0.9828],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.3664, 0.5195, 0.6141, 0.7984],
        [0.8391, 0.5977, 0.1250, 0.2164],
        [0.5336, 0.6078, 0.1516, 0.2094],
        [0.4492, 0.6180, 0.1133, 0.1781],
        [0.6992, 0.6250, 0.1422, 0.1781],
        [0.4609, 0.6367, 0.0812, 0.

## 8. Loss Function

In [ ]:
import torch
import torch.nn as nn

def bbox_iou(box1, box2, eps=1e-7):
    """
    # box1: [N,4] xyxy
    # box2: [N,4] xyxy
    # return: [N,1]
    """
    cx, cy, w, h = box1.unbind(-1)
    x1_b1 = cx - w / 2
    x2_b1 = cx + w / 2
    y1_b1 = cy - h /2 
    y2_b1 = cy + h / 2
    cx, cy, w, h = box2.unbind(-1)
    x1_b2 = cx - w / 2
    x2_b2 = cx + w / 2
    y1_b2 = cy - h /2 
    y2_b2 = cy + h / 2
    inter_x1 = torch.max(x1_b1, x1_b2)
    inter_y1 = torch.max(y1_b1, y1_b2)
    inter_x2 = torch.min(x2_b1, x2_b2)
    inter_y2 = torch.min(y2_b1, y2_b2)
    inter_w = (inter_x2 - inter_x1).clamp(0)
    inter_h = (inter_y2 - inter_y1).clamp(0)
    inter_area = inter_w * inter_h

    area1 = (x2_b1 - x1_b1) * (y2_b1 - y1_b1)
    area2 = (x2_b2 - x1_b2) * (y2_b2 - y1_b2)
    union = area1 + area2 - inter_area + eps

    return (inter_area / union).unsqueeze(-1)


class SingleAssigner(nn.Module):
    def __init__(self, nc, top_k=10, alpha=1.0, beta=6.0, eps=1e-7):
        super().__init__()
        self.nc = nc
        self.top_k = top_k
        self.alpha = alpha
        self.beta = beta
        self.eps = eps

    @torch.no_grad()
    def forward(self, pd_scores, pd_bboxes, anc_points, gt_labels, gt_bboxes, mask_gt):
        """
        Args:
            pd_scores: [num_anchors, num_classes] prediction scores
            pd_bboxes: [num_anchors, 4] xyxy
            anc_points: [num_anchors, 2] anchor centers
            gt_labels: [total_gt, 1] labels
            gt_bboxes: [total_gt, 4] cx,cy,w,h
            batch_idx: [total_gt] batch index for each GT
            cur_batch: int, index of current image
        Returns:
            fg_mask: [num_anchors] bool
            target_bboxes: [num_anchors, 4]
            target_scores: [num_anchors, num_classes]
        """
        # 1. Lấy GT của ảnh hiện tại

        num_anchors = anc_points.shape[0]
        num_max_boxes = gt_bboxes.shape[0]

        if num_max_boxes == 0:
            device = pd_bboxes.device
            return (torch.zeros(num_anchors, dtype=torch.bool, device=device),
                    torch.zeros(num_anchors, 4, device=device),
                    torch.zeros(num_anchors, self.nc, device=device))

        # 2. Chuyển GT sang xyxy
        # cx, cy, w, h = gt_bboxes.unbind(-1)
        # x1 = cx - w / 2
        # x2 = cx + w / 2
        # y1 = cy - h /2 
        # y2 = cy + h / 2
        # gt_bboxes_xyxy = torch.stack([x1, y1, x2, y2], dim=-1)  # [num_max_boxes, 4]

        # 3. mask_in_gts: anchor nằm trong GT
        lt, rb = gt_bboxes[:, None, :2], gt_bboxes[:, None, 2:]  # [num_max_boxes,1,2]
        anc_exp = anc_points[None, :, :]  # [1,num_anchors,2]
        mask_in_gts = torch.cat((anc_exp - lt, rb - anc_exp), dim=-1).amin(-1).gt(self.eps)  # [num_max_boxes,num_anchors]

        # 4. Lặp qua GT để tính IoU và lấy score
        overlaps = torch.zeros(num_max_boxes, num_anchors, device=pd_bboxes.device)
        bbox_scores = torch.zeros(num_max_boxes, num_anchors, device=pd_scores.device)

        for i in range(num_max_boxes):
            pd_boxes = pd_bboxes[mask_in_gts[i]]
            if pd_boxes.numel() == 0:
                # fallback: chọn anchor gần GT center nhất
                gt_center = gt_bboxes[i, :2]
                distances = ((anc_points - gt_center) ** 2).sum(dim=1)
                nearest_idx = distances.argmin()
                mask_in_gts[i, nearest_idx] = True
                pd_boxes = pd_bboxes[nearest_idx:nearest_idx+1]

            gt_boxes = gt_bboxes[i].expand_as(pd_boxes)
            overlaps[i, mask_in_gts[i]] = bbox_iou(pd_boxes, gt_boxes).squeeze(-1).clamp(0)
            cls_id = gt_labels[i].long().item()
            bbox_scores[i, mask_in_gts[i]] = pd_scores[mask_in_gts[i], cls_id]
        # 5. Align metric
        align_metric = bbox_scores.pow(self.alpha) * overlaps.pow(self.beta)

        # 6. Top-k anchor cho mỗi GT
        top_k_metrics, top_k_indices = torch.topk(align_metric, self.top_k, dim=-1)
        mask_top_k = torch.zeros_like(align_metric, dtype=torch.int8)
        ones = torch.ones_like(top_k_indices[:, :1], dtype=torch.int8)
        for k in range(self.top_k):
            mask_top_k.scatter_add_(1, top_k_indices[:, k:k+1], ones)
        mask_top_k.masked_fill_(mask_top_k > 1, 0)
        mask_pos = mask_top_k.float() * mask_in_gts.float()

        # 7. Nếu anchor bị nhiều GT claim
        fg_mask = mask_pos.sum(0) > 0
        if fg_mask.max() > 1:
            max_overlaps_idx = overlaps.argmax(0)
            is_max_overlaps = torch.zeros_like(mask_pos)
            is_max_overlaps.scatter_(0, max_overlaps_idx.unsqueeze(0), 1)
            mask_pos = torch.where(mask_pos.sum(0).unsqueeze(0) > 1, is_max_overlaps, mask_pos).float()
            fg_mask = mask_pos.sum(0) > 0

        # 8. Gán target
        target_gt_idx = mask_pos.argmax(0)
        target_labels = gt_labels[target_gt_idx].clamp(0)
        target_bboxes = gt_bboxes[target_gt_idx]

        target_scores = torch.zeros(num_anchors, self.nc, device=gt_labels.device)
        target_scores.scatter_(1, target_labels.unsqueeze(-1), 1.0)

        # Chuẩn hoá bằng align metric
        pos_align_metrics = align_metric.max(dim=0, keepdim=True)[0]  # max theo GT
        print(pos_align_metrics)
        target_scores = target_scores * pos_align_metrics.T  # [num_anchors,num_classes]
        #fg_mask = mask_pos.sum(0) > 0
        target_scores[~fg_mask] = 0
        target_scores = torch.nan_to_num(target_scores, nan=0.0)
        return fg_mask, target_gt_idx, target_scores


## 9. Training Loop với Resume Capability

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===== Utility functions =====
def cxcywh_to_xyxy(boxes: torch.Tensor) -> torch.Tensor:
    """
    boxes: [N,4] dạng (cx, cy, w, h)
    return: [N,4] dạng (x1, y1, x2, y2)
    """
    cx, cy, w, h = boxes.unbind(-1)
    x1 = cx - w / 2
    y1 = cy - h / 2
    x2 = cx + w / 2
    y2 = cy + h / 2
    return torch.stack([x1, y1, x2, y2], dim=-1)
def make_anchors(featmap_size, stride, device="cpu"):
    """
    Tạo anchor points cho 1 feature map.

    featmap_size: (H, W)
    stride: int (tỉ lệ so với ảnh gốc, vd: 8, 16, 32)
    return: Tensor [H*W, 2]  (cx, cy)
    """
    H, W = featmap_size
    # tạo grid tọa độ
    yv, xv = torch.meshgrid(
        torch.arange(H, device=device),
        torch.arange(W, device=device),
        indexing="ij"
    )
    cx = (xv + 0.5) * stride
    cy = (yv + 0.5) * stride
    anchors = torch.stack((cx, cy), dim=-1).view(-1, 2)  # [H*W, 2]
    return anchors
    
def decode_dfl(pred_dfl, bins=16):
    N = pred_dfl.shape[0]
    pred = pred_dfl.view(N, 4, bins)
    prob = F.softmax(pred, dim=2)
    bin_idx = torch.arange(bins, device=pred.device, dtype=prob.dtype)
    exp_val = (prob * bin_idx.unsqueeze(0).unsqueeze(0)).sum(dim=2)
    return exp_val
def decode_dfl_to_xywh(pred_dfl, anchor_points, bins=16):
    """
    Decode DFL output thành (x, y, w, h)

    pred_dfl: Tensor [N, 4*bins]
    anchor_points: Tensor [N, 2]  # (cx, cy)
    bins: số bins trong DFL
    """
    b, a, c = pred_dfl.shape
    pred = pred_dfl.view(b, a, 4, bins)       # [b,a,4,bins]
    prob = F.softmax(pred, dim=-1)          # softmax theo bins
    bin_idx = torch.arange(bins, device=pred.device, dtype=prob.dtype)
    dist = (prob * bin_idx).sum(dim=-1)     # [b,a,4] -> bxbx(l,t,r,b)

    # anchor points: [a,2] -> [b,a,2]
    cx, cy = anchor_points[:, 0], anchor_points[:, 1]
    cx = cx.unsqueeze(0).expand(b, a)
    cy = cy.unsqueeze(0).expand(b, a)

    l, t, r, b = dist[..., 0], dist[..., 1], dist[..., 2], dist[..., 3]

    # (x1,y1,x2,y2)
    x1 = cx - l
    y1 = cy - t
    x2 = cx + r
    y2 = cy + b

    # (x,y,w,h)
    x = (x1 + x2) / 2
    y = (y1 + y2) / 2
    w = x2 - x1
    h = y2 - y1

    return torch.stack([x, y, w, h], dim=-1)  # [B, HW, 4]
    
def ciou_loss(pred_boxes, target_boxes, eps=1e-7):
    pred_xy1 = pred_boxes[:, :2] - pred_boxes[:, 2:] / 2
    pred_xy2 = pred_boxes[:, :2] + pred_boxes[:, 2:] / 2
    target_xy1 = target_boxes[:, :2] - target_boxes[:, 2:] / 2
    target_xy2 = target_boxes[:, :2] + target_boxes[:, 2:] / 2

    inter_xy1 = torch.max(pred_xy1, target_xy1)
    inter_xy2 = torch.min(pred_xy2, target_xy2)
    inter_wh = (inter_xy2 - inter_xy1).clamp(min=0)
    inter_area = inter_wh[:,0]*inter_wh[:,1]

    pred_area = pred_boxes[:,2]*pred_boxes[:,3]
    target_area = target_boxes[:,2]*target_boxes[:,3]
    union_area = pred_area + target_area - inter_area + eps
    iou = inter_area / union_area

    center_dist = ((pred_boxes[:,:2] - target_boxes[:,:2])**2).sum(dim=1)
    enclose_xy1 = torch.min(pred_xy1, target_xy1)
    enclose_xy2 = torch.max(pred_xy2, target_xy2)
    enclose_diag = ((enclose_xy2 - enclose_xy1)**2).sum(dim=1) + eps

    v = (4/(3.14159265**2)) * torch.pow(
        torch.atan(target_boxes[:,2]/target_boxes[:,3]) - torch.atan(pred_boxes[:,2]/pred_boxes[:,3]), 2
    )
    with torch.no_grad():
        alpha = v / (1 - iou + v + eps)
    ciou = iou - (center_dist / enclose_diag) - alpha * v
    return 1 - ciou

def dfl_loss(preds, targets, bins=16):
    N = targets.shape[0]
    preds = preds.view(N,4,bins)
    loss = 0
    for i in range(4):
        target_val = targets[:, i]
        lower = target_val.floor().clamp(0, bins-2).long()
        upper = lower + 1
        upper_weight = target_val - lower.float()
        lower_weight = 1 - upper_weight
        pred_logits = preds[:, i, :]
        pred_logprob = F.log_softmax(pred_logits, dim=1)
        nll = -(lower_weight*pred_logprob[range(N), lower] +
                upper_weight*pred_logprob[range(N), upper])
        loss += nll.mean()
    return loss

# def bbox_iou(pred_boxes, target_boxes, eps=1e-7):
#     N = pred_boxes.shape[0]
#     M = target_boxes.shape[0]

#     pred_xy1 = pred_boxes[:, :2].unsqueeze(1).expand(N,M,2)
#     pred_xy2 = pred_boxes[:, 2:].unsqueeze(1).expand(N,M,2)
#     target_xy1 = target_boxes[:, :2].unsqueeze(0).expand(N,M,2)
#     target_xy2 = target_boxes[:, 2:].unsqueeze(0).expand(N,M,2)

#     inter_xy1 = torch.max(pred_xy1, target_xy1)
#     inter_xy2 = torch.min(pred_xy2, target_xy2)
#     inter_wh = (inter_xy2 - inter_xy1).clamp(min=0)
#     inter_area = inter_wh[...,0]*inter_wh[...,1]

#     area_pred = ((pred_xy2 - pred_xy1).clamp(min=0)[...,0] *
#                  (pred_xy2 - pred_xy1).clamp(min=0)[...,1])
#     area_target = ((target_xy2 - target_xy1).clamp(min=0)[...,0] *
#                    (target_xy2 - target_xy1).clamp(min=0)[...,1])
#     union_area = area_pred + area_target - inter_area + eps
#     iou = inter_area / union_area
#     return iou

def simota_assigner(pred_cls, pred_boxes, gt_cls, gt_boxes, topk=10):
    N, nc = pred_cls.shape
    M = gt_boxes.shape[0]
    device = pred_cls.device

    if M == 0:
        return torch.zeros(N, dtype=torch.bool, device=device), torch.full((N,), -1, device=device)

    # Classification cost
    gt_onehot = torch.zeros(M, nc, device=device)
    gt_onehot[torch.arange(M), gt_cls] = 1.0
    pred_prob = pred_cls.sigmoid().unsqueeze(1).expand(N,M,nc)
    cls_cost = F.binary_cross_entropy(pred_prob, gt_onehot.unsqueeze(0).expand(N,M,nc), reduction='none').sum(-1)

    # Box cost
    iou_matrix = bbox_iou(pred_boxes, gt_boxes)
    box_cost = 1 - iou_matrix

    total_cost = cls_cost + box_cost

    matched_pred_idx, matched_gt_idx = [], []
    for m in range(M):
        cost = total_cost[:,m]
        topk_val, topk_idx = torch.topk(-cost, min(topk, N))
        matched_pred_idx.append(topk_idx)
        matched_gt_idx.append(torch.full_like(topk_idx, m, device=device))

    matched_pred_idx = torch.cat(matched_pred_idx)
    matched_gt_idx = torch.cat(matched_gt_idx)

    fg_mask = torch.zeros(N, dtype=torch.bool, device=device)
    final_gt_idx = torch.full((N,), -1, device=device)
    for p, g in zip(matched_pred_idx, matched_gt_idx):
        if not fg_mask[p]:
            fg_mask[p] = True
            final_gt_idx[p] = g

    return fg_mask, final_gt_idx

# ===== Full YOLOv8 Loss with SimOTA =====
class YOLOv8Loss(nn.Module):
    def __init__(self, model, strides=[8,16,32], fmap_sizes=[(80,80), (40,40), (20,20)], topk=10):
        super().__init__()
        self.model = model
        self.strides = strides
        self.topk = topk
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='n')
        self.cls_loss = torch.nn.BCEWithLogitsLoss(reduction='none')
        self.fmap_sizes = fmap_sizes
        self.assigner = SingleAssigner(nc=NUM_CLASSES, top_k=10, alpha=0.5, beta=6.0)

    def forward(self, outputs, targets):
        device = outputs[0].device
        cls_targets = targets['cls'].to(device)
        box_targets = targets['box'].to(device)
        idx_targets = targets['idx'].to(device)

        pred_cls_list, pred_box_list, pred_dfl_list = [], [], []
        anchors = []
        # Flatten predictions per feature map
        for i, out in enumerate(outputs):
            b, c, h, w = out.shape
            anch = make_anchors(self.fmap_sizes[i], self.strides[i], device=device)
            anchors.append(anch)
            out = out.permute(0,2,3,1).reshape(b,h*w,c)
            coords = self.model.head.coordinates
            nc = self.model.head.nc
            box_pred = decode_dfl_to_xywh(out[...,:coords], anch)           # [B,H*W,4]
            cls_pred = out[...,coords:coords+nc]  # [B,H*W,nc]
            dfl_pred = out[...,:coords]       # example DFL logits
            pred_box_list.append(box_pred)
            pred_cls_list.append(cls_pred)
            pred_dfl_list.append(dfl_pred)
        anchors_all = torch.cat(anchors, dim=0)
        all_cls_loss, all_box_loss, all_dfl_loss = [], [], []

        for i in range(box_targets.shape[0]):# per batch
            mask = idx_targets == i
            if mask.sum() == 0:
                continue
            t_cls = cls_targets[mask]
            t_box = box_targets[mask]

            # Concatenate multi-scale predictions
            pred_box_i = torch.cat([p[i] for p in pred_box_list], dim=0)
            pred_cls_i = torch.cat([p[i] for p in pred_cls_list], dim=0)
            pred_dfl_i = torch.cat([p[i] for p in pred_dfl_list], dim=0)

            # SimOTA assigner
            pred_box_xyxy = cxcywh_to_xyxy(pred_box_i)
            tag_box_xyxy = cxcywh_to_xyxy(t_box)
            fg_mask, matched_gt_idx, target_scores= self.assigner(pred_cls_i, pred_box_xyxy, anchors_all, t_cls, tag_box_xyxy, mask)
            if fg_mask.sum() == 0:
                continue
            pred_box_fg = pred_box_i[fg_mask]
            pred_cls_fg = pred_cls_i[fg_mask]
            pred_dfl_fg = pred_dfl_i[fg_mask]
            gt_idx = matched_gt_idx[fg_mask]
            target_box_fg = t_box[gt_idx]
            target_cls_fg = t_cls[gt_idx]
            target_scores_fg = target_scores[gt_idx]
            target_scores_sum = max(target_scores_fg.sum(), 1)
            num_fg = fg_mask.sum().clamp(min=1).float()  # số anchor foreground tối thiểu 1
            loss_cls = self.cls_loss(pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / target_scores_sum
            # loss_cls = self.cls_loss(pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / num_fg
            # loss_cls = self.cls_loss(pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / target_scores_sum

            num_fg = fg_mask.sum().clamp(min=1).float()
            loss_cls = self.cls_loss(
                    pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / num_fg
            print(target_scores_fg)
            loss_cls = torch.nan_to_num(loss_cls, nan=0.0)
            # loss_cls = torch.tensor(0., device=pred_cls_fg.device) #= self.cls_loss(pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / target_scores_sum  # BCE
            # print(loss_cls)
            # if target_scores_sum != 0:
            #     loss_cls = self.cls_loss(pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / target_scores_sum
            # # else:
            # #     loss_cls = self.cls_loss(pred_cls_fg, target_scores_fg.to(pred_cls_fg.dtype)).sum() / target_scores_sum
            # # loss_cls = torch.tensor(0.0, dtype=torch.float32, device=pred_box_fg.device)

            # Box loss
            pred_box_decoded = decode_dfl(pred_dfl_fg, bins=16)
            box_loss_val = ciou_loss(pred_box_decoded, target_box_fg).mean()

            # DFL loss
            dfl_loss_val = dfl_loss(pred_dfl_fg, target_box_fg)

            all_cls_loss.append(loss_cls)
            all_box_loss.append(box_loss_val)
            all_dfl_loss.append(dfl_loss_val)

        # if len(all_cls_loss) == 0:
        #     total_loss = torch.tensor(0.0, device=device, requires_grad=True)
        #     box_loss_val = cls_loss = dfl_loss_val = total_loss
        # else:
            #total_loss = torch.stack(all_cls_loss + all_box_loss + all_dfl_loss).sum()
        box_loss_val = torch.stack(all_box_loss).mean()
        cls_loss = torch.stack(all_cls_loss).mean()
        dfl_loss_val = torch.stack(all_dfl_loss).mean()
        box_loss_val *= 7.5 
        cls_loss *= 0.5
        dfl_loss_val *= 1.5
        total_loss = box_loss_val + cls_loss + dfl_loss_val

        return {"total_loss":total_loss,
                "box_loss":box_loss_val,
                "cls_loss":cls_loss,
                "dfl_loss":dfl_loss_val}


In [14]:
import torch
from torchvision.ops import box_iou

def compute_metrics_batch_map50_95(outputs, targets, num_classes=5, device='cuda'):
    """
    Compute precision, recall, F1, mAP50, mAP50-95 per batch (multi-class, YOLOv8)
    
    outputs: list of dicts with 'boxes', 'scores', 'labels' for each image
    targets: list of dicts with 'boxes', 'labels' for each image
    """
    iou_thresholds = [0.5 + i*0.05 for i in range(10)]  # 0.5,0.55,...,0.95
    
    batch_precisions, batch_recalls, batch_f1s, batch_map50, batch_map50_95 = [], [], [], [], []
    
    for i in range(len(outputs)):
        pred_boxes = outputs[i]['boxes'].to(device)
        pred_scores = outputs[i]['scores'].to(device)
        pred_labels = outputs[i]['labels'].to(device)
        
        true_boxes = targets[i]['boxes'].to(device)
        true_labels = targets[i]['labels'].to(device)
        
        all_results = {c: {'tp': [], 'fp': [], 'scores': [], 'num_gt': 0} for c in range(num_classes)}
        
        # tách theo class
        for c in range(num_classes):
            mask_pred = pred_labels == c
            mask_gt = true_labels == c
            
            preds_c = pred_boxes[mask_pred]
            scores_c = pred_scores[mask_pred]
            gts_c = true_boxes[mask_gt]
            
            all_results[c]['num_gt'] += len(gts_c)
            
            if len(preds_c) == 0:
                all_results[c]['tp'].append(torch.zeros(0, device=device))
                all_results[c]['fp'].append(torch.zeros(0, device=device))
                all_results[c]['scores'].append(torch.zeros(0, device=device))
                continue
            
            ious = box_iou(preds_c, gts_c) if len(gts_c) > 0 else torch.zeros((len(preds_c),0), device=device)
            assigned_gt = torch.full((len(gts_c),), -1, dtype=torch.long, device=device)
            
            tp = torch.zeros(len(preds_c), device=device)
            fp = torch.zeros(len(preds_c), device=device)
            
            for j in range(len(preds_c)):
                if ious.size(1) == 0:
                    fp[j] = 1
                    continue
                max_iou, max_idx = ious[j].max(0)
                if max_iou >= 0.5 and assigned_gt[max_idx] == -1:
                    tp[j] = 1
                    assigned_gt[max_idx] = j
                else:
                    fp[j] = 1
            
            all_results[c]['tp'].append(tp)
            all_results[c]['fp'].append(fp)
            all_results[c]['scores'].append(scores_c)
        
        # compute per class metrics
        precisions, recalls, f1s, ap50_list, ap50_95_list = [], [], [], [], []
        for c in range(num_classes):
            if all_results[c]['num_gt'] == 0:
                precisions.append(0)
                recalls.append(0)
                f1s.append(0)
                ap50_list.append(0)
                ap50_95_list.append(0)
                continue
            
            tp = torch.cat(all_results[c]['tp'])
            fp = torch.cat(all_results[c]['fp'])
            scores = torch.cat(all_results[c]['scores'])
            
            sorted_idx = torch.argsort(scores, descending=True)
            tp_cum = torch.cumsum(tp[sorted_idx], dim=0)
            fp_cum = torch.cumsum(fp[sorted_idx], dim=0)
            
            precision = tp_cum / (tp_cum + fp_cum + 1e-16)
            recall = tp_cum / (all_results[c]['num_gt'] + 1e-16)
            f1 = 2 * precision * recall / (precision + recall + 1e-16)
            
            precisions.append(precision[-1].item() if len(precision)>0 else 0)
            recalls.append(recall[-1].item() if len(recall)>0 else 0)
            f1s.append(f1[-1].item() if len(f1)>0 else 0)
            
            # AP50 và AP50-95
            ap_per_iou = []
            for t in iou_thresholds:
                tp_iou = (tp_cum * (tp_cum / (tp_cum+fp_cum+1e-16) >= t).float())
                recall_curve = tp_iou / (all_results[c]['num_gt']+1e-16)
                precision_curve = tp_iou / (tp_iou + fp_cum + 1e-16)
                
                ap = 0
                for r in torch.linspace(0,1,11):
                    p = precision_curve[recall_curve>=r].max() if (recall_curve>=r).any() else 0
                    ap += p / 11
                ap_per_iou.append(ap.item())
            
            ap50_list.append(ap_per_iou[0])         # IoU=0.5
            ap50_95_list.append(sum(ap_per_iou)/len(ap_per_iou))  # trung bình IoU=0.5:0.95
        
        batch_precisions.append(sum(precisions)/num_classes)
        batch_recalls.append(sum(recalls)/num_classes)
        batch_f1s.append(sum(f1s)/num_classes)
        batch_map50.append(sum(ap50_list)/num_classes)
        batch_map50_95.append(sum(ap50_95_list)/num_classes)
    
    return batch_precisions, batch_recalls, batch_f1s, batch_map50, batch_map50_95

## 10. Bắt đầu/Resume Training

In [15]:
def find_latest_checkpoint(checkpoint_dir):
    """
    Tìm checkpoint mới nhất để resume training
    """
    checkpoint_files = []
    if os.path.exists(checkpoint_dir):
        for f in os.listdir(checkpoint_dir):
            if f.startswith('checkpoint_epoch_') and f.endswith('.pth'):
                epoch_num = int(f.split('_')[2].split('.')[0])
                checkpoint_files.append((epoch_num, os.path.join(checkpoint_dir, f)))

    if checkpoint_files:
        # Sắp xếp theo epoch và lấy checkpoint mới nhất
        checkpoint_files.sort(key=lambda x: x[0])
        return checkpoint_files[-1][1]
    return None

def save_checkpoint(model, optimizer, scheduler, epoch, train_loss, val_loss, checkpoint_dir, is_best=False):
    """
    Save checkpoint để có thể resume training
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'num_classes': NUM_CLASSES,
        'model_version': MODEL_VERSION,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE
    }

    # Save regular checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
    torch.save(checkpoint, checkpoint_path)

    # Save best model
    if is_best:
        best_path = os.path.join(checkpoint_dir, 'best_model.pth')
        torch.save(checkpoint, best_path)
        # Cũng save vào main save path
        torch.save(checkpoint, SAVE_PATH)

    return checkpoint_path

def train_model():
    if train_loader is None:
        print("Error: Dataset not loaded.")
        return

    model = CustomYolo(version=MODEL_VERSION, num_classes=NUM_CLASSES)
    model.to(device)

    criterion = YOLOv8Loss(model)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.0005)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

    train_losses = []
    train_box_losses = []
    train_cls_losses = []
    train_dfl_losses = []
    val_losses = []
    val_box_losses = []
    val_cls_losses = []
    val_dfl_losses = []
    lr_history = [] 
    best_val_loss = float('inf')
    start_epoch = 0
    history = []
    HISTORY_FILE = os.path.join(CHECKPOINT_DIR, "training_history.csv")
    # Resume training if checkpoint exists
    # if RESUME_TRAINING:
    #     latest_checkpoint = find_latest_checkpoint(CHECKPOINT_DIR)
    #     if latest_checkpoint:
    #         checkpoint = torch.load(latest_checkpoint, map_location=device)
    #         model.load_state_dict(checkpoint['model_state_dict'])
    #         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #         scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    #         start_epoch = checkpoint['epoch'] + 1
    #         best_val_loss = checkpoint.get('val_loss', float('inf'))
    #         train_losses = checkpoint.get('train_losses', [])
    #         val_losses = checkpoint.get('val_losses', [])

    # # Training loop
    for epoch in range(start_epoch, NUM_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        train_box_loss = 0.0
        train_cls_loss = 0.0
        train_dfl_loss = 0.0 
        
        train_batches = 0

        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Train')
        for batch_idx, (images, targets) in enumerate(train_pbar):
            images = images.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            losses = criterion(outputs, targets)
            loss = losses["total_loss"] #tensor
            train_box_loss += losses["box_loss"]
            train_cls_loss += losses["cls_loss"]
            train_dfl_loss += losses["dfl_loss"]
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_batches += 1
            train_pbar.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Box_loss': f"{losses['box_loss']:.4f}",
                'Cls_loss': f"{losses['cls_loss']:.4f}",
                'Dfl_loss': f"{losses['dfl_loss']:.4f}"
            })

            if device.type == 'cuda':
                torch.cuda.empty_cache()

        # Validation phase
        #model.eval()
        val_loss = 0.0
        val_box_loss = 0.0 
        val_cls_loss = 0.0
        val_dfl_loss = 0.0 
        val_batches = 0

        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Val')
            for images, targets in val_pbar:
                images = images.to(device)
                outputs = model(images)
                losses = criterion(outputs, targets)
                loss = losses['total_loss']
                val_loss += loss.item()
                val_box_loss += losses["box_loss"]
                val_cls_loss += losses["cls_loss"]
                val_dfl_loss += losses["dfl_loss"]
                val_batches += 1
                val_pbar.set_postfix({
                    'Loss': f"{loss.item():.4f}",
                    'Box_loss': f"{losses['box_loss']:.4f}",
                    'Cls_loss': f"{losses['cls_loss']:.4f}",
                    'Dfl_loss': f"{losses['dfl_loss']:.4f}"
                })

        
        avg_train_loss = train_loss / max(train_batches, 1)
        avg_train_box_loss = train_box_loss / max(train_batches, 1)
        avg_train_cls_loss = train_cls_loss / max(train_batches, 1)
        avg_train_dfl_loss = train_dfl_loss / max(train_batches, 1)
        avg_val_loss = val_loss / max(val_batches, 1)
        avg_val_box_loss = val_box_loss / max(val_batches, 1)
        avg_val_cls_loss = val_cls_loss / max(val_batches, 1)
        avg_val_dfl_loss = val_dfl_loss / max(val_batches, 1)
        train_losses.append(avg_train_loss)
        train_box_losses.append(avg_train_box_loss)
        train_cls_losses.append(avg_train_cls_loss)
        train_dfl_losses.append(avg_train_dfl_loss)
        val_losses.append(avg_val_loss)
        val_box_losses.append(avg_val_box_loss)
        val_cls_losses.append(avg_val_cls_loss)
        val_dfl_losses.append(avg_val_dfl_loss)
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        lr_history.append(current_lr)
        is_best = avg_val_loss < best_val_loss
        
        history.append({
            "epoch": epoch,
            "train_loss": avg_train_loss,
            "train_box_loss": avg_train_box_loss,
            "train_cls_loss": avg_train_cls_loss,
            "train_dfl_loss": avg_train_dfl_loss,
            "val_loss": avg_val_loss,
            "val_box_loss": avg_val_box_loss,
            "val_cls_loss": avg_val_cls_loss,
            "val_dfl_loss": avg_val_dfl_loss,
        })
        file_exists = os.path.isfile(HISTORY_FILE)
        # with open(HISTORY_FILE, mode="a", newline="") as f:
        #     writer = csv.writer(f)
        #     # Ghi header nếu file mới tạo
        #     if not file_exists:
        #         writer.writerow([
        #             "epoch", "train_loss", "train_box_loss", "train_cls_loss", "train_dfl_loss",
        #             "val_loss","val_box_loss", "val_cls_loss", "val_dfl_loss"
        #             ])
        #     writer.writerow([
        #         epoch, train_loss, train_box_loss, train_cls_loss, train_dfl_loss, 
        #         val_loss, val_box_loss, val_cls_loss, val_dfl_loss])
        if is_best:
            best_val_loss = avg_val_loss

        # Save checkpoint
        if (epoch + 1) % SAVE_CHECKPOINT_EVERY == 0 or is_best or epoch == NUM_EPOCHS - 1:
            save_checkpoint(model, optimizer, scheduler, epoch, avg_train_loss, avg_val_loss, CHECKPOINT_DIR, is_best)

        print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train: {avg_train_loss:.4f}, Val: {avg_val_loss:.4f}, LR: {current_lr:.6f}")

        if device.type == 'cuda':
            torch.cuda.empty_cache()

    # Plot training curves
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(range(start_epoch, start_epoch + len(train_losses)), train_losses, label='Train Loss', color='blue')
    plt.plot(range(start_epoch, start_epoch + len(val_losses)), val_losses, label='Val Loss', color='red')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    # epochs_range = range(start_epoch, start_epoch + len(train_losses))
    # lr_values = [LEARNING_RATE * (0.5 ** (i // 30)) for i in epochs_range]
    # plt.plot(epochs_range, lr_values)
    # plt.title('Learning Rate Schedule')
    # plt.xlabel('Epoch')
    # plt.ylabel('Learning Rate')
    # plt.grid(True)

    plt.subplot(1, 3, 3)
    plt.plot(range(start_epoch, start_epoch + len(train_losses)), lr_history)
    plt.title("Learning Rate")
    plt.xlabel("Epoch")
    plt.ylabel("LR")
    plt.grid(True)
    plt.show()

    if len(val_losses) > 1:
        improvement = [(val_losses[0] - val_loss) / val_losses[0] * 100 for val_loss in val_losses]
        plt.plot(range(start_epoch, start_epoch + len(improvement)), improvement)
        plt.title('Validation Loss Improvement (%)')
        plt.xlabel('Epoch')
        plt.ylabel('Improvement (%)')
        plt.grid(True)

    plt.tight_layout()
    plt.show()

    return model, train_losses, val_losses

In [ ]:
# Start training
if train_loader is not None:
    trained_model, train_losses, val_losses = train_model()
else:
    print("Error: Dataset not loaded properly. Please check dataset path and structure.")

Epoch 1/10 - Train:   0%|          | 0/281 [00:00<?, ?it/s]

tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[1.5345e-15, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0')
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5345e-15]],
       device='cuda:0')
tensor([[1.0583e-14, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0')
tensor([[0.0000e+00, 1.0583e-14, 0.0000e+00, 0.0000e+00, 0.0000e+00]],
       device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[1.7585e-14, 0.0000e+00, 0.0000e+00, 

Epoch 1/10 - Train:   0%|          | 1/281 [00:03<18:06,  3.88s/it, Loss=29.3778, Box_loss=10.5071, Cls_loss=1.7249, Dfl_loss=17.1459]

tensor([[1.7991e-14, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0')
tensor([[1.7991e-14, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],
       device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]],

Epoch 1/10 - Train:   1%|          | 2/281 [00:04<09:44,  2.10s/it, Loss=28.2656, Box_loss=10.4632, Cls_loss=1.6694, Dfl_loss=16.1330]

...

Epoch 1/10 - Train:  93%|█████████▎| 262/281 [03:51<00:16,  1.15it/s, Loss=7.9031, Box_loss=3.8172, Cls_loss=0.0000, Dfl_loss=4.0859]

tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0

Epoch 1/10 - Train:  94%|█████████▎| 263/281 [03:52<00:15,  1.14it/s, Loss=8.1803, Box_loss=4.0640, Cls_loss=0.0000, Dfl_loss=4.1163]

tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.]], device='cuda:0')
tensor([[nan, 0., 0.,  ..., 0., 0., 0

Epoch 1/10 - Train:  94%|█████████▎| 263/281 [03:53<00:15,  1.14it/s, Loss=7.9668, Box_loss=3.8851, Cls_loss=0.0001, Dfl_loss=4.0816]

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Tổng số tham số: {total_params:,}")

In [ ]:
import torch
from torchvision.ops import nms

def postprocess_yolo_sigmoid(pred_boxes, pred_cls, score_thr=0.5, iou_thr=0.5):
    """
    Postprocess YOLO-like outputs (dùng sigmoid cho multi-label).
    
    Args:
        pred_boxes: Tensor [N, 4]  - toạ độ box dạng (x1, y1, x2, y2)
        pred_cls:   Tensor [N, num_classes] - logits cho từng class (chưa sigmoid)
        score_thr:  float - ngưỡng lọc confidence
        iou_thr:    float - ngưỡng NMS
        
    Returns:
        final_boxes:  Tensor [M, 4]
        final_scores: Tensor [M]
        final_labels: Tensor [M]
    """
    # 1. Sigmoid để lấy prob cho từng class
    probs = torch.sigmoid(pred_cls)  # [N, num_classes]

    # 2. Lấy những (box, class) có score > threshold
    num_boxes, num_classes = probs.shape
    boxes_all, scores_all, labels_all = [], [], []

    for c in range(num_classes):
        scores_c = probs[:, c]
        mask = scores_c > score_thr
        if mask.sum() == 0:
            continue
        boxes_c = pred_boxes[mask]
        scores_c = scores_c[mask]
        labels_c = torch.full((scores_c.shape[0],), c, dtype=torch.long, device=pred_boxes.device)

        # 3. NMS riêng cho từng class
        keep = nms(boxes_c, scores_c, iou_thr)
        boxes_all.append(boxes_c[keep])
        scores_all.append(scores_c[keep])
        labels_all.append(labels_c[keep])

    if len(boxes_all) == 0:
        return torch.empty((0, 4)), torch.empty((0,)), torch.empty((0,), dtype=torch.long)

    # 4. Ghép lại
    final_boxes = torch.cat(boxes_all, dim=0)
    final_scores = torch.cat(scores_all, dim=0)
    final_labels = torch.cat(labels_all, dim=0)

    return final_boxes, final_scores, final_labels

In [ ]:
import torch
from torchvision.ops import nms

def postprocess_yolo_sigmoid_batch(pred_boxes, pred_cls, score_thr=0.5, iou_thr=0.5):
    """
    Postprocess YOLO-like outputs for a batch (multi-label sigmoid).
    
    Args:
        pred_boxes: Tensor [B, N, 4]  - (x1, y1, x2, y2)
        pred_cls:   Tensor [B, N, num_classes] - logits cho từng class (chưa sigmoid)
        score_thr:  float - ngưỡng confidence
        iou_thr:    float - ngưỡng NMS
        
    Returns:
        results: list of length B
            mỗi phần tử là dict {
                "boxes": Tensor [M, 4],
                "scores": Tensor [M],
                "labels": Tensor [M]
            }
    """
    B, N, num_classes = pred_cls.shape
    results = []

    for b in range(B):
        boxes = pred_boxes[b]       # [N, 4]
        logits = pred_cls[b]        # [N, num_classes]
        probs = torch.sigmoid(logits)

        boxes_all, scores_all, labels_all = [], [], []

        for c in range(num_classes):
            scores_c = probs[:, c]
            mask = scores_c > score_thr
            if mask.sum() == 0:
                continue

            boxes_c = boxes[mask]
            scores_c = scores_c[mask]
            labels_c = torch.full((scores_c.shape[0],), c, dtype=torch.long, device=boxes.device)

            keep = nms(boxes_c, scores_c, iou_thr)

            boxes_all.append(boxes_c[keep])
            scores_all.append(scores_c[keep])
            labels_all.append(labels_c[keep])

        if len(boxes_all) == 0:
            results.append({
                "boxes": torch.empty((0, 4), device=boxes.device),
                "scores": torch.empty((0,), device=boxes.device),
                "labels": torch.empty((0,), dtype=torch.long, device=boxes.device)
            })
        else:
            results.append({
                "boxes": torch.cat(boxes_all, dim=0),
                "scores": torch.cat(scores_all, dim=0),
                "labels": torch.cat(labels_all, dim=0)
            })

    return results


## 11. Load Trained Model

In [ ]:
def load_trained_model(model_path):
    if not os.path.exists(model_path):
        return None

    checkpoint = torch.load(model_path, map_location=device)
    model = CustomYolo(
        version=checkpoint.get('model_version', MODEL_VERSION),
        num_classes=checkpoint.get('num_classes', NUM_CLASSES)
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    return model

# if os.path.exists(SAVE_PATH):
#     loaded_model = load_trained_model(SAVE_PATH)
# else:
#     loaded_model = None

## 12. Inference Function

In [ ]:
def inference_single_image(model, image_path, conf_threshold=0.5):
    if model is None:
        return None

    image = cv2.imread(image_path)
    if image is None:
        return None

    original_image = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    h, w = image.shape[:2]
    scale = INPUT_SIZE / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)

    image = cv2.resize(image, (new_w, new_h))
    padded_image = np.full((INPUT_SIZE, INPUT_SIZE, 3), 114, dtype=np.uint8)
    padded_image[:new_h, :new_w] = image

    image_tensor = torch.from_numpy(padded_image.transpose(2, 0, 1)).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)

    return outputs, original_image

# # Test inference if model and dataset available
# if loaded_model is not None and 'val_images' in locals() and len(val_images) > 0:
#     result = inference_single_image(loaded_model, val_images[0])

## 13. Summary và Hướng dẫn sử dụng cho Google Colab

### 🚀 Tính năng chính của phiên bản Google Colab:

#### ✅ **Resume Training**
- Tự động tìm và load checkpoint mới nhất
- Tiếp tục training từ đúng epoch đã dừng
- Giữ nguyên optimizer state và learning rate schedule
- Set `RESUME_TRAINING = True` để kích hoạt

#### ✅ **Google Colab Optimization**
- Auto mount Google Drive
- Optimized memory management
- GPU detection và configuration
- Paths tự động cho Colab environment

#### ✅ **No Data Augmentation**
- Dataset class đã được simplified
- Không có horizontal flip hay augmentation khác
- Chỉ resize và padding cơ bản

### 📝 Các bước sử dụng:

#### **Bước 1: Setup Environment**
```python
# Chạy cell đầu tiên để mount Google Drive
# Sẽ tự động detect GPU và setup paths
```

#### **Bước 2: Cấu hình Dataset**
```python
ROBOFLOW_API_KEY = "your_actual_api_key"
WORKSPACE_NAME = "your-workspace"
PROJECT_NAME = "your-project"
NUM_CLASSES = 5  # Số classes trong dataset của bạn
```

#### **Bước 3: Download Dataset**
```python
# Uncomment code trong download_roboflow_dataset()
# Dataset sẽ được download vào /content/roboflow_dataset
```

#### **Bước 4: Start/Resume Training**
```python
# Uncomment trong cell "Bắt đầu Training"
trained_model, train_losses, val_losses = train_model()
```

### 🔧 **Resume Training Workflow**

#### Nếu Colab disconnect trong khi training:
1. **Reconnect** và chạy lại từ đầu đến cell cấu hình
2. **Set RESUME_TRAINING = True** (mặc định đã True)  
3. **Chạy lại train_model()** - sẽ tự động tìm checkpoint mới nhất
4. **Training sẽ tiếp tục** từ epoch đã save

#### Checkpoint Management:
- **Auto-save** mỗi 10 epochs (configurable)
- **Best model** được save khi val loss giảm
- **Checkpoints** lưu trong Google Drive (persistent)
- **Manual resume** từ epoch cụ thể nếu cần

### 💾 **File Structure trên Google Drive**
```
/content/drive/MyDrive/YOLOv8_Training/
├── custom_yolo_model.pth          # Best model
├── checkpoints/
│   ├── checkpoint_epoch_10.pth    # Regular checkpoints
│   ├── checkpoint_epoch_20.pth
│   ├── best_model.pth             # Copy of best
│   └── ...
└── logs/ (optional)
```

### ⚡ **Performance Tips cho Colab**

#### Memory Management:
- **Batch size**: Start với 16, giảm xuống 8 nếu OOM
- **num_workers**: Set 2 cho dataloaders  
- **pin_memory**: True khi dùng GPU
- **torch.cuda.empty_cache()**: Auto cleanup sau mỗi epoch

#### Training Interruption:
- **Google Colab timeout**: ~12 hours max
- **Save frequency**: Mỗi 10 epochs (có thể adjust)
- **Best model**: Always saved khi có improvement
- **Resume**: Seamless với RESUME_TRAINING=True

### 🐛 **Troubleshooting**

#### Common Issues:
- **Out of Memory**: Giảm BATCH_SIZE xuống 8 hoặc 4
- **Dataset not found**: Check DATASET_PATH và folder structure  
- **Resume không work**: Check CHECKPOINT_DIR có checkpoints không
- **Slow training**: Verify GPU được sử dụng (`device = cuda`)

#### Debug Commands:
```python
# Check GPU status
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

# Check checkpoint
latest = find_latest_checkpoint(CHECKPOINT_DIR)
print(f"Latest checkpoint: {latest}")

# Check dataset
print(f"Train images: {len(train_images)}")
print(f"Val images: {len(val_images)}")
```

### 🎯 **Training Time Estimates**
Với dataset 12,000 samples, 5 classes:
- **Google Colab GPU (T4)**: ~4-6 giờ cho 100 epochs  
- **Google Colab CPU**: ~50-70 giờ (không khuyến nghị)
- **Colab Pro GPU (A100/V100)**: ~2-3 giờ cho 100 epochs

### 📊 **Monitoring Training**
- Progress bars hiển thị realtime loss
- Training curves plot sau khi hoàn thành  
- Checkpoint saves được log ra console
- Best model được highlight khi tìm thấy

---
### 🚦 **Ready to Train!**
Tất cả đã setup xong. Uncomment dòng training trong cell 10 để bắt đầu!